**Preparing the data pipeline**

In [1]:
import sys
import os
import numpy as np 
import pandas as pd 
import math
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = 'white')
# sns.set(style='whitegrid', color_codes=True)
from tqdm import tqdm
import plotly
import time
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as pyo
import warnings
import random
import gc
# import dask.dataframe as dd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import validation
import re
from sys import getsizeof
import scipy
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt
from itertools import cycle
from pandas import Grouper
from scipy.sparse import csr_matrix
import joblib
from sklearn import preprocessing
import lightgbm as lgbm
from sklearn import metrics
import joblib
from sklearn.preprocessing import OneHotEncoder
plt.style.use('ggplot')
pd.set_option('display.max_columns', None)
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Dense,Flatten,BatchNormalization,Dropout,LeakyReLU,LSTM,Conv1D,TimeDistributed,MaxPool1D,Flatten,Input
from tensorflow.keras.regularizers import l1,l2,l1_l2
import tensorflow.keras.initializers as initializer
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler
import json
tf.random.set_seed(120)
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
# rfered this blog
# https://pythonhosted.org/calmap/
# np.random.seed(sum(map(ord, 'calmap')))
import calmap
import pickle
import joblib
# from joblib import Parallel,delayed
from scipy.stats import norm
from sklearn.feature_extraction.text import TfidfVectorizer
color_cycle = cycle(["blue","green","red","cyan","magenta",'yellow','black','darkorange','grey','lime'])
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
tf.executing_eagerly() 

True

**Let's store some preprocessed data like lable encoder**

In [3]:
def reduce_mem_usage(df):
  incoming_df_mem_usage = df.memory_usage().sum() / 1024**2
  print("Memory usage of properties dataframe is :",incoming_df_mem_usage," MB")
  na_list = [] # to keep track the columns that have missing values filled in
  for col in df.columns:
    if df[col].dtype != object:
      is_int = False
      max =  df[col].max()
      min = df[col].min()

      # integer doesn't support nan, so it needs to be filled
      if not np.isfinite(df[col].all()):
        na_list.app(col)
        df[col].fillna(mn-1,inplace=True)
      # test if column can be converted to integer
      asint = df[col].fillna(0).astype(np.int64)
      result = (df[col] - asint)
      result = result.sum()

      if result > -0.01 and result < 0.01:
        is_int = True
      if is_int:
        if min>=0:
          if max < 255:
            df[col] = df[col].astype(np.uint8)
          elif max < 65535:
            df[col] = df[col].astype(np.uint16)
          elif max < 4294967295:
            df[col] = df[col].astype(np.uint32)
          else:
            df[col] = df[col].astype(np.uint64)
        else:
          if min > np.iinfo(np.int8).min and max < np.iinfo(np.int8).max:
            df[col] = df[col].astype(np.int8)
          elif min > np.iinfo(np.int16).min and max < np.iinfo(np.int16).max:
            df[col] = df[col].astype(np.int16)
          elif min > np.iinfo(np.int32).min and max < np.iinfo(np.int32).max:
            df[col] = df[col].astype(np.int32)
          elif min > np.iinfo(np.int64).min and max < np.iinfo(np.int64).max:
            df[col] = df[col].astype(np.int64)
      else:
        df[col] = df[col].astype(np.float32)
         
  print("********************** MEMORY USAGE AFTER COMPLETION **********************")
  outgoing_df_mem_usage = df.memory_usage().sum() / 1024**2 
  print("Memory usage is: ",outgoing_df_mem_usage," MB")
  print("This is ",100*outgoing_df_mem_usage/incoming_df_mem_usage,"% of the initial size")
  return df, na_list

In [4]:
sales_train_evaluation = pd.read_csv(r"sales_train_evaluation.csv")  #The properties dataset
sales_train_evaluation, na_list = reduce_mem_usage(sales_train_evaluation)
if(len(na_list)!=0):
  print("="*100)
  print("Note: The following columns have missed values filled in with data_frame['column_name'].min() -1")
  print("="*100)
  print(na_list)

Memory usage of properties dataframe is : 452.91172790527344  MB
********************** MEMORY USAGE AFTER COMPLETION **********************
Memory usage is:  65.71533966064453  MB
This is  14.509524839327831 % of the initial size


In [5]:
calendar = pd.read_csv(r"calendar.csv")  #The properties dataset
calendar, na_list = reduce_mem_usage(calendar)
if(len(na_list)!=0):
  print("="*100)
  print("Note: The following columns have missed values filled in with data_frame['column_name'].min() -1")
  print("="*100)
  print(na_list)

Memory usage of properties dataframe is : 0.2104339599609375  MB
********************** MEMORY USAGE AFTER COMPLETION **********************
Memory usage is:  0.1221780776977539  MB
This is  58.06005728373577 % of the initial size


In [6]:
sell_prices = pd.read_csv(r"sell_prices.csv")  #The properties dataset
sell_prices, na_list = reduce_mem_usage(sell_prices)
if(len(na_list)!=0):
  print("="*100)
  print("Note: The following columns have missed values filled in with data_frame['column_name'].min() -1")
  print("="*100)
  print(na_list)

Memory usage of properties dataframe is : 208.77456665039062  MB
********************** MEMORY USAGE AFTER COMPLETION **********************
Memory usage is:  143.5325527191162  MB
This is  68.75001827184856 % of the initial size


**We have data for 1941 days, first 1913 for train dataset and 1914-1941 is validation dataset and we have to predict the sales for next 28 days so, we have add d_42 to d_1970 with values and make it as our test dataset**


In [7]:
for d in range(1942,1970):
    col = 'd_' + str(d)
    sales_train_evaluation[col] = 0
    sales_train_evaluation[col] = sales_train_evaluation[col].astype(np.int16)

In [8]:
# day_cols = ['id','item_id','dept_id','cat_id','store_id','state_id']
# for d in range(1970-180,1970):
#     col = 'd_' + str(d)
#     day_cols.append(col)
# sales_train_evaluation_test = sales_train_evaluation[day_cols]

In [9]:
df = pd.melt(sales_train_evaluation, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='units_sold').dropna()

In [10]:
df = pd.merge(df, calendar, on='d', how='left')
df = pd.merge(df, sell_prices, on=['store_id','item_id','wm_yr_wk'], how='left') 

In [11]:
df['date'] = pd.to_datetime(df['date'])
df['day_of_month'] = df['date'].dt.day.astype('int8')
df['week_of_month'] = df['date'].apply(lambda d: (d.day-1) // 7 + 1).astype('int8')
df['week_of_year'] = df['date'].dt.week.astype('int8')
df['quarter_of_year'] = df['date'].dt.quarter.astype('int8')

In [12]:
df['is_month_start'] = [1 if i == 1 else 0 for i in df['week_of_month']]
df['is_month_end'] = [1 if i in [4,5] else 0 for i in df['week_of_month']]
df['event_name_1'] = df['event_name_1'].fillna(0)
df['event_name_2'] = df['event_name_2'].fillna(0)
df['event_type_1'] = df['event_type_1'].fillna(0)
df['event_type_2'] = df['event_type_2'].fillna(0)
df['is_week_end'] = [1 if i in [1,2] else 0 for i in df['wday']]
df['close_to_week_days'] = [1 if i in [3,7] else 0 for i in df['wday']]
df['mid_weeK_days'] = [1 if i in [4,5,6] else 0 for i in df['wday']]
df['is_weekday'] = [1 if i not in [1,2] else 0 for i in df['wday']]

In [13]:
df = df.drop(['date','wm_yr_wk','weekday'],axis=1)

In [2]:
df.to_pickle('best_model/data_level1.pkl')
del df
gc.collect()

In [9]:
df = pd.read_pickle('best_model/data_level1.pkl')

In [4]:
def label_encoder(values,category_name):
    le = preprocessing.LabelEncoder()
    le.fit(values)
    output = le.transform(values)
    joblib.dump(le,f'best_model/preprocessed/{category_name}.pkl')
    return output

In [5]:
df['item_id'] = label_encoder(df['item_id'].values,'item_id_le')
df['dept_id'] = label_encoder(df['dept_id'].values,'dept_id_le')
df['cat_id'] = label_encoder(df['cat_id'].values,'cat_id_le')
df['state_id'] = label_encoder(df['state_id'].values,'state_id_le')
df['store_id'] = label_encoder(df['store_id'].values,'store_id_le')

In [8]:
cat_df = reduce_mem_usage(df[['item_id','dept_id','cat_id','state_id','store_id']])
df = df.drop(['item_id','dept_id','cat_id','state_id','store_id'],axis=1)
gc.collect();

Memory usage of properties dataframe is : 2748.175506591797  MB
********************** MEMORY USAGE AFTER COMPLETION **********************
Memory usage is:  801.5511894226074  MB
This is  29.166666666666668 % of the initial size


In [9]:
df.shape

(60034810, 24)

In [13]:
df['event_name_1'] = label_encoder(df['event_name_1'].astype(str).values,'event_name_1_le')
df['event_name_2'] = label_encoder(df['event_name_2'].astype(str).values,'event_name_2_le')
df['event_type_1'] = label_encoder(df['event_type_1'].astype(str).values,'event_type_1_le')
df['event_type_2'] = label_encoder(df['event_type_2'].astype(str).values,'event_type_2_le')
event_df = reduce_mem_usage(df[['event_name_1','event_name_2','event_type_1','event_type_2']])
df = df.drop(['event_name_1','event_name_2','event_type_1','event_type_2'],axis=1)
df_test = pd.concat([cat_df[0],event_df[0]],axis=1)
df = pd.concat([df,df_test],axis=1)
del df_test
gc.collect()

Memory usage of properties dataframe is : 2290.146255493164  MB
********************** MEMORY USAGE AFTER COMPLETION **********************
Memory usage is:  687.0438766479492  MB
This is  30.0 % of the initial size


0

In [22]:
df['units_sold'] = df['units_sold'].fillna(0)

In [6]:
df['store_item_sold_avg']  = df.groupby(['store_id','item_id'])['units_sold'].transform('mean').astype(np.float16)

In [7]:
avg_store_item_cols = ['store_id','item_id','store_item_sold_avg']
avg_store_item_df = df[avg_store_item_cols]
avg_store_item_df.to_pickle(f'best_model/preprocessed/avg_store_item_df.pkl')

In [10]:
avg_store_item_df = pd.read_pickle('best_model/preprocessed/avg_store_item_df.pkl')
avg_store_item_df = avg_store_item_df.drop_duplicates()
avg_store_item_df.to_pickle(f'best_model/preprocessed/avg_store_item_df_unique.pkl')

In [30]:
other_info = {'median_sell_price':str(df['sell_price'].median())}
with open('best_model/preprocessed/other_info.json','w') as outfile:
    json.dump(other_info,outfile)

In [31]:
other_info

{'median_sell_price': '3.47'}